# TODO

- улучшить парсер (убрать пустые тексты)
- убрать наны из текстов!!!!
- передвинуть CV после метрик


## Метрики
+ + длина текста
+ + max min tf-idf
+ 0 max и min косинусное расстояние между парами слов (соседних и не соседних, если успеем посчитать), tf-idf взвешенное (умножать каждое слово на tf-idf этого слова для этого документа),
____________
+ вытащить пять самых частотных слов из текста (слово с самым высоким tf-idf - классификация с помощью правил в зависимости от пересечения самых значимых слов для группы со словами в данном документе)
+ средний вектор текста сравнить со средними по каждой группе (взять центроид текста и сравнить с центроидом группы)
+ то же самое, только по пяти частотным словам

## Модели
+ логистическая регрессия
+ случайный лес (выкинуть матрицы текстов)
+ [xgboost](https://www.kaggle.com/stuarthallows/using-xgboost-with-scikit-learn)
+ классификация до и после понижения размерности фичей (только все инты) с помощью Principal Composition Analysis

! делать конфьюжен матрикс

sns.heatmap(data=confusion_matrix(twenty_test.target, predicted), annot=True, fmt="d", cbar=False, xticklabels=categories, yticklabels=categories)

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import matplotlib
%matplotlib inline
import random
import warnings

df = pd.read_csv('data.csv', index_col='id')
df = df.sample(frac=1)
df.head()

,text,group
id,,
2688,КУДАХ И ТОРНАДО\n\nочень скучаю по торнадо. ме...,kudah
2997,"В КУДАХЕ ТОННЕЛЬ, НО НЕ ДЛЯ ПОБЕГА\n\nкудах оп...",kudah
1414,NaN,telegony
1752,NaN,telegony
931,NaN,krishnaits


In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
df.describe(include=[np.object])

,text,group
count,2447,3000
unique,2429,5
top,Доброе утро!,podkoren
freq,5,600


In [5]:
import nltk
import re
from pymystem3 import Mystem

nltk.download('punkt')
mystem_analyzer = Mystem()

lemmatized = []
unlemmatized = []

for doc in df['text']:
    text = re.sub("<br>", "", str(doc))
    word_list = nltk.word_tokenize(text)
    unlemmatized.append(word_list)
    lemmatized.append([mystem_analyzer.lemmatize(word)[0] for word in word_list])

[nltk_data] Downloading package punkt to /home/valeria/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
from itertools import chain

count_unlem = 0

for el in unlemmatized:
    count_unlem += len(el)

cont_lemmatized = list(chain.from_iterable(lemmatized))

print("Количество токенов в датасете:", count_unlem)
print("Количество уникальных лемм:", len(set(cont_lemmatized)))

Количество токенов в датасете: 692543
Количество уникальных лемм: 37248


In [7]:
words_per_doc = np.mean([len(text) for text in unlemmatized])
symbols_per_doc = np.mean([len(str(doc)) for doc in df["text"]])

print("Среднее количество слов на документ:", int(words_per_doc))
print("Среднее количество символов на документ:", int(symbols_per_doc))

Среднее количество слов на документ: 230
Среднее количество символов на документ: 1281


In [8]:
from nltk.corpus import stopwords
import pymorphy2 as pm2

pmm = pm2.MorphAnalyzer()
russian_stopwords = stopwords.words("russian")

# сюда добавить парсинг получше
def parse(text):
    text = re.sub(r'[^\w\s]','', str(text))
    text = re.sub("[0-9A-Za-z#_=]", "", text)
    text = [pmm.normal_forms(x)[0] for x in text.split() if x not in russian_stopwords]
    if text == [np.nan]:
        return(['nan'])
    return text

df['text'] = df['text'].apply(parse)

In [9]:
df.head(5)

,text,group
id,,
2688,"[куд, и, торнадо, очень, скучать, торнадо, хле...",kudah
2997,"[в, кудах, тоннель, но, не, для, побег, куд, с...",kudah
1414,[],telegony
1752,[],telegony
931,[],krishnaits


## TF-IDF

In [12]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess

dct = Dictionary()

dct.add_documents(list(df["text"]))
corpus = []
for idx, row in df.iterrows():
    corpus.append(dct.doc2bow(row['text']))
tf_idf = TfidfModel(corpus)

In [13]:
vectors = [tf_idf[doc] for doc in corpus]

In [14]:
vectors = [{dct[el[0]]: el[1] for el in vector} for vector in vectors]
vectors[0]

{'антропология': 0.13193552536887726,
 'большой': 0.05344479063789524,
 'братец': 0.15032721936852053,
 'вдохнуть': 0.1552589573836702,
 'весь': 0.09518281651471602,
 'вечность': 0.12109074706973376,
 'воскликнуть': 0.12866292143201075,
 'вряд': 0.0921973472432415,
 'вывод': 0.08963642679724876,
 'выполнить': 0.09274880429267256,
 'выпустить': 0.10163798396933127,
 'глубоко': 0.09776731505494081,
 'говориться': 0.08528866133929136,
 'грань': 0.11552077925803443,
 'должный': 0.047119136143471956,
 'дорогой': 0.07444388304014787,
 'ещё': 0.03700306878934491,
 'закончиться': 0.09923202700494284,
 'знать': 0.042512466224095914,
 'и': 0.030564587722126505,
 'иметь': 0.15479264270676557,
 'исследование': 0.09574388920952373,
 'история': 0.05175899247409029,
 'итог': 0.15131470121113363,
 'конец': 0.05233391859077387,
 'корень': 0.16205688399197954,
 'который': 0.057524361579211584,
 'кто': 0.05415846801491459,
 'культурный': 0.08451657979140888,
 'литература': 0.07831455195453832,
 'любезный

## Embeddings

In [88]:
!wget "http://corpus.leeds.ac.uk/tools/ru/ruwac-parsed.out.xz"

--2019-06-06 23:14:42--  http://corpus.leeds.ac.uk/tools/ru/ruwac-parsed.out.xz
Распознаётся corpus.leeds.ac.uk (corpus.leeds.ac.uk)… 129.11.80.147
Подключение к corpus.leeds.ac.uk (corpus.leeds.ac.uk)|129.11.80.147|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 9768295304 (9,1G) [application/x-xz]
Сохранение в: «ruwac-parsed.out.xz.1»

ruwac-parsed.out.xz   0%[                    ]   6,39M  1,41MB/s    ост 2h 8m  ^C


In [15]:
import lzma
import os
from collections import Counter
from datetime import datetime
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LsiModel
from progressbar import progressbar
from sklearn import svm
from sklearn.exceptions import UndefinedMetricWarning
from sklearn import metrics 
from sklearn.model_selection import LeaveOneOut, cross_val_score, GridSearchCV, train_test_split
from string import punctuation as punct
from string import ascii_letters as latin
from tqdm import tqdm
punct += '»«'

corpus_archive_filename = 'ruwac-parsed.out.xz'
corpus_filename = 'corpus.txt'
premodel_filename = 'small.model'
vocabulary_filename = 'freq.txt'
model_filename = 'full.model'
sample_directory = 'sample'
manual_filename = 'manual_metrics.csv'
df_filename = 'data.csv'
lsi_model_small_filename = 'small_lsi.model'
wordlist_filename = 'wordlist.json'
idfs_filename = 'idfs.json'

In [ ]:
with lzma.open(corpus_archive_filename, mode='rt', encoding='utf-8') as f:
    with open(corpus_filename, mode='w', encoding='utf-8') as t:
        current = []
        for line_number, line in enumerate(f):
            if line_number % 10000000 == 0:
                print(line_number, datetime.now().time(), file=open('archive_log.txt', 'a', encoding='utf-8'))
            l = line.split('\t')
            if len(l) == 7:
                lemm = l[3]
                sent = l[1]
                if sent == 'SENT':
                    if set(latin).intersection(set(''.join(current))) == set():
                        current.append('\n')
                        print(' '.join(current), file=t, end='')
                    current = []
                elif lemm not in punct:
                    current.append(lemm)

In [16]:
from gensim.models import Word2Vec

In [17]:
sentences = list(df["text"])

In [18]:
premodel = Word2Vec(sentences, sg=1, min_count=1, workers=1, size=100)

In [20]:
list(premodel.wv.vocab.items())[0]

('любезный', <gensim.models.keyedvectors.Vocab at 0x7fc0745843c8>)

In [21]:
premodel.similarity('сестричка', 'убить')

0.92137635

In [147]:
addition = "\n"

for sent in sentences:
    addition += " ".join(sent) + '\n'
    
with open(corpus_filename, mode='a', encoding='utf-8') as t:
    t.write(addition)

In [22]:
def get_vocabulary(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            for word in line.split():
                yield word

def get_sentences(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            yield line.split()

In [23]:
from collections import Counter

word_freq = Counter(tqdm(get_vocabulary(corpus_filename)))

487263005it [04:24, 1842969.96it/s]


In [24]:
model = Word2Vec(sg=1, min_count=1, workers=1, size=100)

In [25]:
model.build_vocab_from_freq(word_freq)

In [26]:
model.train(get_sentences(corpus_filename), epochs=1, total_examples=len(word_freq))

(414160863, 487263005)

In [27]:
model.save("new_model")

In [28]:
list(model.wv.vocab.items())[:10]

[('это', <gensim.models.keyedvectors.Vocab at 0x7fc010fa0400>),
 ('тип', <gensim.models.keyedvectors.Vocab at 0x7fbff74fe0b8>),
 ('девушка', <gensim.models.keyedvectors.Vocab at 0x7fbff74fe2e8>),
 ('в', <gensim.models.keyedvectors.Vocab at 0x7fbff74fe048>),
 ('один', <gensim.models.keyedvectors.Vocab at 0x7fbff74fe438>),
 ('день', <gensim.models.keyedvectors.Vocab at 0x7fbff74fe2b0>),
 ('отмечать', <gensim.models.keyedvectors.Vocab at 0x7fbff74fe160>),
 ('что', <gensim.models.keyedvectors.Vocab at 0x7fbff74fe748>),
 ('на', <gensim.models.keyedvectors.Vocab at 0x7fbff74fe1d0>),
 ('себя', <gensim.models.keyedvectors.Vocab at 0x7fbff74fe278>)]

In [31]:
model.similarity('сестричка', 'бля')

0.8574129

In [32]:
model.wv["антропология"]

array([ 0.39728817,  0.33759257,  0.7590469 ,  0.06518362, -0.00707845,
        0.21875107, -0.278317  , -0.12087888, -0.8043595 ,  0.21836177,
       -0.1006732 ,  0.3203989 , -0.51800084,  0.4067611 , -0.22997738,
       -0.16281371,  0.1696374 ,  0.15048559,  0.21203   , -0.24017143,
       -0.69083244,  0.49337438,  0.24594243, -0.14898317, -0.30758995,
        0.43765056, -0.15040557, -0.26634857, -0.18923329, -0.21882819,
        0.31550246, -0.10189518, -0.26420575,  0.03930803, -0.5591204 ,
        0.20390655, -0.10754047, -0.46035388,  0.3988782 ,  0.66853565,
        0.12194882,  0.20048839,  0.03152714,  0.22004724,  0.22344092,
        0.08621364,  0.08817811,  0.41683516,  0.33218208, -0.2985513 ,
       -0.06771833,  0.00605447,  0.36615172,  0.2530866 , -0.77041537,
       -0.14882922, -0.09573819, -0.05361597, -0.24468362,  0.34957165,
       -0.05098194, -0.8742342 , -0.05177836,  0.5112704 , -0.2002943 ,
       -0.19358756,  0.31097212, -0.49275833, -0.12766156, -0.07

In [33]:
df['weighted_vec'] = pd.Series(np.nan, index=df.index)
df.head()

,text,group,weighted_vec
id,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,NaN
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,NaN
2,"[за, время, существование, наш, сообщество, со...",podkoren,NaN
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,NaN
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,NaN


In [47]:
defect_words = 0

def get_embs(word):
    global defect_words
    try:
        word_emb = model.wv[re.sub("ё", "е", str(word))]
    except KeyError:
        word_emb = np.zeros(100)
        defect_words += 1
    return word_emb

def doc_to_vec(doc, vector):
    vec = [get_embs(word) * vector[word] for word in doc]
    return vec

defect_words

0

In [48]:
d2v = [doc_to_vec(df["text"][doc], vectors[doc]) for doc in range(len(df["weighted_vec"]))]
df['weighted_vec'] = pd.Series(d2v, index=df.index)

In [49]:
df.head()

,text,group,weighted_vec,word_length,symbol_length,max_idf,min_idf
id,,,,,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,"[[-0.024145605, 0.11946748, 0.261655, -0.06556...",107,701,0.357202,0.026378
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,"[[-0.05039152, 0.057766065, 0.103294566, -0.01...",161,1062,0.384330,0.015935
2,"[за, время, существование, наш, сообщество, со...",podkoren,"[[-0.05420818, 0.03600782, 0.068977825, -0.006...",118,829,0.220884,0.020886
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,"[[-0.027693467, 0.072877966, 0.056998827, -0.0...",240,1712,0.213519,0.017622
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,"[[-0.053390473, 0.04924318, 0.08710171, 0.0078...",233,1519,0.425739,0.020320


In [50]:
df['word_length'] = pd.Series([len(doc) for doc in df["text"]], index=df.index)
df.head()

,text,group,weighted_vec,word_length,symbol_length,max_idf,min_idf
id,,,,,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,"[[-0.024145605, 0.11946748, 0.261655, -0.06556...",107,701,0.357202,0.026378
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,"[[-0.05039152, 0.057766065, 0.103294566, -0.01...",161,1062,0.384330,0.015935
2,"[за, время, существование, наш, сообщество, со...",podkoren,"[[-0.05420818, 0.03600782, 0.068977825, -0.006...",118,829,0.220884,0.020886
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,"[[-0.027693467, 0.072877966, 0.056998827, -0.0...",240,1712,0.213519,0.017622
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,"[[-0.053390473, 0.04924318, 0.08710171, 0.0078...",233,1519,0.425739,0.020320


In [51]:
def symbol_length(doc):
    length = 0
    for word in doc:
        length += len(word)
    return length

df['symbol_length'] = pd.Series([symbol_length(doc) for doc in df["text"]], index=df.index)
df.head()

,text,group,weighted_vec,word_length,symbol_length,max_idf,min_idf
id,,,,,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,"[[-0.024145605, 0.11946748, 0.261655, -0.06556...",107,701,0.357202,0.026378
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,"[[-0.05039152, 0.057766065, 0.103294566, -0.01...",161,1062,0.384330,0.015935
2,"[за, время, существование, наш, сообщество, со...",podkoren,"[[-0.05420818, 0.03600782, 0.068977825, -0.006...",118,829,0.220884,0.020886
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,"[[-0.027693467, 0.072877966, 0.056998827, -0.0...",240,1712,0.213519,0.017622
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,"[[-0.053390473, 0.04924318, 0.08710171, 0.0078...",233,1519,0.425739,0.020320


In [52]:
def max_idf(vec):
    ret = -2
    for key in vec:
        if vec[key] > ret:
            ret = vec[key]
    return ret

def min_idf(vec):
    ret = 2
    for key in vec:
        if vec[key] < ret:
            ret = vec[key]
    return ret

df['max_idf'] = pd.Series([max_idf(vec) for vec in vectors], index=df.index)
df['min_idf'] = pd.Series([min_idf(vec) for vec in vectors], index=df.index)

df.head()

,text,group,weighted_vec,word_length,symbol_length,max_idf,min_idf
id,,,,,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,"[[-0.024145605, 0.11946748, 0.261655, -0.06556...",107,701,0.357202,0.026378
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,"[[-0.05039152, 0.057766065, 0.103294566, -0.01...",161,1062,0.384330,0.015935
2,"[за, время, существование, наш, сообщество, со...",podkoren,"[[-0.05420818, 0.03600782, 0.068977825, -0.006...",118,829,0.220884,0.020886
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,"[[-0.027693467, 0.072877966, 0.056998827, -0.0...",240,1712,0.213519,0.017622
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,"[[-0.053390473, 0.04924318, 0.08710171, 0.0078...",233,1519,0.425739,0.020320


Here we will redifine cosine similarity because we are retarded

In [53]:
def cos_sim(v1, v2):
    return np.inner(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [68]:
def local_sym(vector_matrix):
    sym_list = []
    for i in range(len(vector_matrix)):
        if i != 0:
            sym_list.append(cos_sim(vector_matrix[i-1], vector_matrix[i]))
    return sym_list

def crutched_max(vec):
    if len(vec) == 0:
        return 0
    else:
        return max(vec)

def crutched_mean(vec):
    if len(vec) < 2:
        return 0
    else:
        return np.nanmean(vec)

In [69]:
df['max_loc_sym'] = pd.Series([crutched_max(local_sym(vec)) for vec in df['weighted_vec']], index=df.index)
df['mean_loc_sym'] = pd.Series([crutched_mean(local_sym(vec)) for vec in df['weighted_vec']], index=df.index)

df.head()

,text,group,weighted_vec,word_length,symbol_length,max_idf,min_idf,max_loc_sym,mean_loc_sym
id,,,,,,,,,
0,"[любезный, братец, сестричка, под, корень, зак...",podkoren,"[[-0.024145605, 0.11946748, 0.261655, -0.06556...",107,701,0.357202,0.026378,0.946426,0.841490
1,"[завтра, воскресение, сообщество, появиться, п...",podkoren,"[[-0.05039152, 0.057766065, 0.103294566, -0.01...",161,1062,0.384330,0.015935,0.946426,0.765385
2,"[за, время, существование, наш, сообщество, со...",podkoren,"[[-0.05420818, 0.03600782, 0.068977825, -0.006...",118,829,0.220884,0.020886,0.939897,0.774720
3,"[жизнь, продолжаться, продолжаться, дорога, на...",podkoren,"[[-0.027693467, 0.072877966, 0.056998827, -0.0...",240,1712,0.213519,0.017622,1.000000,0.767779
4,"[на, день, погибнуть, наш, автор, любимый, бра...",podkoren,"[[-0.053390473, 0.04924318, 0.08710171, 0.0078...",233,1519,0.425739,0.020320,0.946426,0.766841


In [70]:
source2id = {'podkoren': 0, 
             'telegony': 1,
             'kudah': 2,
             'ufo': 3, 
             'krishnaits': 4}

id2source = {v:k for k, v in source2id.items()}
df["group"] = df["group"].replace(source2id)

df.head()

,text,group,weighted_vec,word_length,symbol_length,max_idf,min_idf,max_loc_sym,mean_loc_sym
id,,,,,,,,,
0,"[любезный, братец, сестричка, под, корень, зак...",0,"[[-0.024145605, 0.11946748, 0.261655, -0.06556...",107,701,0.357202,0.026378,0.946426,0.841490
1,"[завтра, воскресение, сообщество, появиться, п...",0,"[[-0.05039152, 0.057766065, 0.103294566, -0.01...",161,1062,0.384330,0.015935,0.946426,0.765385
2,"[за, время, существование, наш, сообщество, со...",0,"[[-0.05420818, 0.03600782, 0.068977825, -0.006...",118,829,0.220884,0.020886,0.939897,0.774720
3,"[жизнь, продолжаться, продолжаться, дорога, на...",0,"[[-0.027693467, 0.072877966, 0.056998827, -0.0...",240,1712,0.213519,0.017622,1.000000,0.767779
4,"[на, день, погибнуть, наш, автор, любимый, бра...",0,"[[-0.053390473, 0.04924318, 0.08710171, 0.0078...",233,1519,0.425739,0.020320,0.946426,0.766841


In [83]:
unvec_data = df.drop(columns=['text', 'weighted_vec'])
unvec_data.head()

,group,word_length,symbol_length,max_idf,min_idf,max_loc_sym,mean_loc_sym
id,,,,,,,
0,0,107,701,0.357202,0.026378,0.946426,0.841490
1,0,161,1062,0.384330,0.015935,0.946426,0.765385
2,0,118,829,0.220884,0.020886,0.939897,0.774720
3,0,240,1712,0.213519,0.017622,1.000000,0.767779
4,0,233,1519,0.425739,0.020320,0.946426,0.766841


In [84]:
# possibly absolutely useless but I am retarded 

vec_data = df[['weighted_vec','group']]
vec_data.head()

,weighted_vec,group
id,,
0,"[[-0.024145605, 0.11946748, 0.261655, -0.06556...",0
1,"[[-0.05039152, 0.057766065, 0.103294566, -0.01...",0
2,"[[-0.05420818, 0.03600782, 0.068977825, -0.006...",0
3,"[[-0.027693467, 0.072877966, 0.056998827, -0.0...",0
4,"[[-0.053390473, 0.04924318, 0.08710171, 0.0078...",0


In [76]:
def CV(df):
    cross_val_folds = []
    fold = []
    for idx, row in df.iterrows():
        fold.append(row)
        if idx % (len(df) // 10) == 0:
            cross_val_folds.append(fold)
            fold = []
    return cross_val_folds

In [87]:
cross_val_folds = CV(unvec_data)[1:]

In [88]:
for el in cross_val_folds:
    print(len(el))

300
300
300
300
300
300
300
300
300


# Classification